In [1]:
import pandas as pd # library for data analysis
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML documents

# get the response in the form of html
wikiurl="https://en.wikipedia.org/wiki/Template:COVID-19_pandemic_death_rates"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
print(response.status_code)

200


In [2]:
soup = BeautifulSoup(response.text, 'html.parser')
casetable=soup.find('table',{'class':"wikitable"})
df=pd.read_html(str(casetable))
# convert list to dataframe
cases_df=pd.DataFrame(df[0])
print(cases_df)

              Country  Confirmed cases  Deaths Case fatality rate  \
0             Belgium           632321   18939               3.0%   
1          San Marino             2205      56               2.5%   
2            Slovenia           109934    2454               2.2%   
3               Italy          1991278   70395               3.5%   
4                Peru          1000153   37218               3.7%   
..                ...              ...     ...                ...   
166  Papua New Guinea              761       9               1.2%   
167          Thailand             5762      60               1.0%   
168          Tanzania              509      21               4.1%   
169           Vietnam             1421      35               2.5%   
170            Taiwan              776       7               0.9%   

     Deaths per 100,000 population  
0                           165.81  
1                           165.75  
2                           118.70  
3                      

In [3]:
local_outbreak_items = pd.read_csv("reference.csv")

In [4]:
reconciled = cases_df.merge(local_outbreak_items, left_on="Country", right_on="countryLabel").drop_duplicates()

In [5]:
set(cases_df["Country"]) - set(reconciled["Country"])

{'Bahamas',
 'China',
 'Congo',
 'DR Congo',
 'Gambia',
 'Palestine',
 'United States'}

In [6]:
template_to_label = {
    'Bahamas': 'The Bahamas',
    'China': 'mainland China',
    'Congo': 'Republic of the Congo',
    'DR Congo': 'Democratic Republic of the Congo',
    'Gambia': 'The Gambia',
    'Palestine': 'State of Palestine',
    'United States': 'United States of America' 
}

In [7]:
cases_df.Country = cases_df.Country.replace(template_to_label)

In [8]:
reconciled = cases_df.merge(local_outbreak_items, left_on="Country", right_on="countryLabel").drop_duplicates()

In [9]:
set(cases_df["Country"]) - set(reconciled["Country"])

set()

In [10]:
reconciled

Country  Confirmed cases  Deaths Case fatality rate  \
0             Belgium           632321   18939               3.0%   
1          San Marino             2205      56               2.5%   
2            Slovenia           109934    2454               2.2%   
3               Italy          1991278   70395               3.5%   
4                Peru          1000153   37218               3.7%   
..                ...              ...     ...                ...   
169  Papua New Guinea              761       9               1.2%   
170          Thailand             5762      60               1.0%   
171          Tanzania              509      21               4.1%   
172           Vietnam             1421      35               2.5%   
173            Taiwan              776       7               0.9%   

     Deaths per 100,000 population       item      countryLabel  \
0                           165.81  Q84446340           Belgium   
1                           165.75  Q87123561        San Marino   
2                           118.70  Q87250948          Slovenia   
3                           116.49  Q84104992             Italy   
4                           116.35  Q87587763              Peru   
..                             ...        ...               ...   
169                           0.10  Q88137634  Papua New Guinea   
170                           0.09  Q83873566          Thailand   
171                           0.04  Q87770827          Tanzania   
172                           0.04  Q83873057           Vietnam   
173                           0.03  Q84081307            Taiwan   

                                      itemLabel  
0             2020 COVID-19 pandemic in Belgium  
1          2020 COVID-19 pandemic in San Marino  
2                 COVID-19 pandemic in Slovenia  
3                    COVID-19 pandemic in Italy  
4                     COVID-19 pandemic in Peru  
..                                          ...  
169  2020 COVID-19 pandemic in Papua New Guinea  
170          2020 COVID-19 pandemic in Thailand  
171          2020 COVID-19 pandemic in Tanzania  
172                COVID-19 pandemic in Vietnam  
173                 COVID-19 pandemic in Taiwan  

[174 rows x 8 columns]

In [11]:
reconciled["Case fatality rate"] = [float(i.replace("%",""))/100 for i in reconciled["Case fatality rate"]]

In [12]:
reconciled

Country  Confirmed cases  Deaths  Case fatality rate  \
0             Belgium           632321   18939               0.030   
1          San Marino             2205      56               0.025   
2            Slovenia           109934    2454               0.022   
3               Italy          1991278   70395               0.035   
4                Peru          1000153   37218               0.037   
..                ...              ...     ...                 ...   
169  Papua New Guinea              761       9               0.012   
170          Thailand             5762      60               0.010   
171          Tanzania              509      21               0.041   
172           Vietnam             1421      35               0.025   
173            Taiwan              776       7               0.009   

     Deaths per 100,000 population       item      countryLabel  \
0                           165.81  Q84446340           Belgium   
1                           165.75  Q87123561        San Marino   
2                           118.70  Q87250948          Slovenia   
3                           116.49  Q84104992             Italy   
4                           116.35  Q87587763              Peru   
..                             ...        ...               ...   
169                           0.10  Q88137634  Papua New Guinea   
170                           0.09  Q83873566          Thailand   
171                           0.04  Q87770827          Tanzania   
172                           0.04  Q83873057           Vietnam   
173                           0.03  Q84081307            Taiwan   

                                      itemLabel  
0             2020 COVID-19 pandemic in Belgium  
1          2020 COVID-19 pandemic in San Marino  
2                 COVID-19 pandemic in Slovenia  
3                    COVID-19 pandemic in Italy  
4                     COVID-19 pandemic in Peru  
..                                          ...  
169  2020 COVID-19 pandemic in Papua New Guinea  
170          2020 COVID-19 pandemic in Thailand  
171          2020 COVID-19 pandemic in Tanzania  
172                COVID-19 pandemic in Vietnam  
173                 COVID-19 pandemic in Taiwan  

[174 rows x 8 columns]

In [13]:
from wikidataintegrator import wdi_core, wdi_login
from wikidataintegrator.wdi_helpers import try_write
import os
import pandas as pd
import pprint
from IPython.display import clear_output
from getpass import getpass

WBUSER = "TiagoLubiana"  
WBPASS = getpass(prompt='Enter your password: ')  
login = wdi_login.WDLogin(WBUSER, WBPASS)


Enter your password:  ············


https://www.wikidata.org/w/api.php
Successfully logged in as TiagoLubiana


In [14]:
from datetime import datetime

today_in_wikidata_format = datetime.today().strftime('+%Y-%m-%dT00:00:00Z')
today_in_wikidata_format

'+2020-12-24T00:00:00Z'

In [15]:
statements = []
for i, row in reconciled.iterrows():
    s = "Q102044164"
    p = "P527"
    o = row["item"]
    q1 = "P1120"
    oq1 = row["Deaths"]
    q2 = "P1603"
    oq2 = row["Confirmed cases"]
    q3 = "P3457"
    oq3 = row["Case fatality rate"]
    q4 = "P585"
    oq4 = today_in_wikidata_format
    r1 = "P854"
    or1 = "https://coronavirus.jhu.edu/data/mortality"
    r2 = "P143"
    or2 = "Q328"
    r3 = "P4656"
    or3 = "https://en.wikipedia.org/wiki/Template:COVID-19_pandemic_death_rates"


    qualifier_list = [wdi_core.WDQuantity(oq1, q1,  is_qualifier=True),
                  wdi_core.WDQuantity(oq2, q2,  is_qualifier=True),
                  wdi_core.WDQuantity(oq3, q3,  is_qualifier=True), 
                  wdi_core.WDTime(oq4, q4,  is_qualifier=True)]
    
    reference_list = [[wdi_core.WDUrl(or1, r1, is_reference=True),
                  wdi_core.WDItemID(or2, r2, is_reference=True),
                  wdi_core.WDUrl(or3, r3, is_reference=True)]]
    
    statements.extend([wdi_core.WDItemID(value= o, prop_nr=p, qualifiers=qualifier_list, references=reference_list)])
    
item = wdi_core.WDItemEngine(wd_item_id=s, data=statements)   
item.write(login)

'Q102044164'